In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
data = load_breast_cancer()
X = data.data
Y = data.target

In [5]:
print(X)
print(Y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(569, 30) (455, 30) (114, 30)


In [7]:
scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)

X_test_std  = scaler.transform(X_test)


In [8]:
# convert data to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train_std, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test_std, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

In [9]:
type(X_train)

torch.Tensor

In [10]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [11]:
# define hyperparameter

input_size = X_train.shape[1]
hidden_size = 10
output_size = 1
num_epochs = 100
batch_size = 20
learning_rate = 0.01

In [12]:
# initialise neural network and move it to the GPU

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [13]:
# define loss and optimizer

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# training the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

    # calculate acccuracy
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == Y_train.view(-1, 1)).float().sum()
        accuracy = correct / Y_train.size(0)
    if((epoch+1) % 10 == 0):
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss: 0.3386, Accuracy: 92.53%
Epoch [20/100], Loss: 0.1656, Accuracy: 94.07%
Epoch [30/100], Loss: 0.1053, Accuracy: 96.92%
Epoch [40/100], Loss: 0.0812, Accuracy: 98.02%
Epoch [50/100], Loss: 0.0681, Accuracy: 98.46%
Epoch [60/100], Loss: 0.0598, Accuracy: 98.68%
Epoch [70/100], Loss: 0.0546, Accuracy: 98.68%
Epoch [80/100], Loss: 0.0500, Accuracy: 98.68%
Epoch [90/100], Loss: 0.0459, Accuracy: 98.68%
Epoch [100/100], Loss: 0.0420, Accuracy: 98.90%


In [15]:
# evaluation on training set

model.eval()
with torch.no_grad():
    outputs = model(X_train)
    predicted = outputs.round()
    correct = (predicted == Y_train.view(-1, 1)).float().sum()
    accuracy = correct / Y_train.size(0)
    print(f"Training Accuracy: {accuracy.item() * 100:.2f}%")

Training Accuracy: 98.90%


In [16]:
# evaluation on training set

model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == Y_test.view(-1, 1)).float().sum()
    accuracy = correct / Y_test.size(0)
    print(f"Training Accuracy: {accuracy.item() * 100:.2f}%")

Training Accuracy: 95.61%


In [1]:
!pip install ultralytics

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 342.2 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 342.2 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 342.2 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 342.2 kB/s eta 0:00:0